# **Estudo de Caso 1: ETL**

Este Jupyter Notebook demonstra um fluxo completo de **ETL** (Extract, Transform, Load) com as seguintes etapas principais:

---

## **1. Extração (Extract)**
Nesta etapa, os dados são extraídos de um dataset "sujo" disponibilizado no **Kaggle**. Essa fonte contém informações que precisam de limpeza e padronização para uso.

---

## **2. Transformação (Transform)**
O dataset passa por um processo de tratamento e transformação, incluindo:
- **Limpeza de dados**: Remoção de valores nulos, inconsistências e outliers.
- **Transformações estruturais**: Ajustes de formato e normalização de colunas.
- **Enriquecimento**: Inclusão de informações adicionais ou calculadas.

As transformações são realizadas com **Python**, utilizando bibliotecas como:
- `Pandas`: Para manipulação de dados em tabelas.
- `NumPy`: Para cálculos e operações numéricas.

---

## **3. Carregamento (Load)**
O dataset tratado é carregado em um banco de dados **PostgreSQL**, tornando os dados acessíveis para análises posteriores e consultas investigativas.

---

### **Ferramentas e Tecnologias Utilizadas**
- **Python**: Linguagem de programação para automação das etapas de ETL.
- **Pandas e NumPy**: Bibliotecas essenciais para manipulação e transformação de dados.
- **PostgreSQL**: Banco de dados relacional para armazenamento e consulta de dados limpos.
- **Jupyter Notebook**: Ambiente interativo para execução do código e visualização dos resultados.

---

### **Objetivo**
Este estudo de caso demonstra as etapas práticas de um fluxo ETL, preparando dados brutos para uso eficiente em análises e tomada de decisão.


## Extração e Transformação 

In [211]:
# import de libs necessárias para o processo de data cleaning

import numpy as np
import pandas as pd
import zipfile
import os
from datetime import datetime

import time
start_time = time.time()

In [212]:
# diretório onde o dataset sera salvo
dataset_dir = "datasets"
os.makedirs(dataset_dir, exist_ok=True)

# link do dataset no kaggle
kaggle_dataset = "yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring"

# baixando o dataset
os.system(f"kaggle datasets download -d {kaggle_dataset} -p {dataset_dir}")

print(f"Dataset baixado e salvo em: {dataset_dir}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring
License(s): CC0-1.0
fifa-21-messy-raw-dataset-for-cleaning-exploring.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset baixado e salvo em: datasets


In [213]:
# caminho do arquivo ZIP baixado
zip_file_path = os.path.join("datasets", "fifa-21-messy-raw-dataset-for-cleaning-exploring.zip")

# extraindo conteúdo
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/Users/gms4/Desktop/guilherme/TCC/Project")


In [214]:
# transformando csv em df e dando uma olhada nele
df = pd.read_csv("fifa21 raw data v2.csv")
df.head()

/var/folders/md/55dn7z6j7cv20tmwqbwsfbjc0000gq/T/ipykernel_70599/3679858239.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fifa21 raw data v2.csv")


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [215]:
# mostrando as colunas
# várias colunas que fogem do padrão de banco de dados
# principalmente as colunas com espaço, como GK Diving e GK Handling
# além de caracteres extras, como ↓OVA
# e espaços ao invés de underline

df.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [216]:
# padronizando as colunas: nome de coluna com letra minuscula e espaço vira _
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('/', '')
df.columns

Index(['id', 'name', 'longname', 'photourl', 'playerurl', 'nationality', 'age',
       '↓ova', 'pot', 'club', 'contract', 'positions', 'height', 'weight',
       'preferred_foot', 'bov', 'best_position', 'joined', 'loan_date_end',
       'value', 'wage', 'release_clause', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'wf', 'sm', 'aw', 'dw', 'ir', 'pac', 'sho', 'pas', 'dri',
       'def', 'ph

In [217]:
print(df.isna().sum()) # valores faltantes por coluna

duplicated_ids = df["id"].duplicated().sum()
print(f"IDs duplicados: {duplicated_ids}") # total de duplicados

id              0
name            0
longname        0
photourl        0
playerurl       0
             ... 
pas             0
dri             0
def             0
phy             0
hits         2595
Length: 77, dtype: int64
IDs duplicados: 0


In [218]:
# dropando colunas de url
df = df.drop(["longname","photourl","playerurl"],axis=1)

# renomeando coluna ↓OVA por causa do caractere especial
df= df.rename(columns={"↓ova":"ova"})

In [219]:
# notando problemática na coluna Club
df["club"].head()

0           \n\n\n\nFC Barcelona
1               \n\n\n\nJuventus
2        \n\n\n\nAtlético Madrid
3        \n\n\n\nManchester City
4    \n\n\n\nParis Saint-Germain
Name: club, dtype: object

In [220]:
# fazendo o strip dos \n
df["club"]=df["club"].str.strip("\r\n\r\n\r\n\r\n")

df["club"].head()

0           FC Barcelona
1               Juventus
2        Atlético Madrid
3        Manchester City
4    Paris Saint-Germain
Name: club, dtype: object

In [221]:
# notando que as alturas não são inteiros, que possuem -cm- no fim
print(df['height'].head())

# fazendo o strip de -cm-
# df["height"]=df["height"].str.strip("cm")

# analisando valores únicos
# perceba valores como 5\'11" e 6\'4", que representam foot e inches
# criaremos uma função para transformar tudo em cm
df["height"].unique()

0    170cm
1    187cm
2    188cm
3    181cm
4    175cm
Name: height, dtype: object


array(['170cm', '187cm', '188cm', '181cm', '175cm', '184cm', '191cm',
       '178cm', '193cm', '185cm', '199cm', '173cm', '168cm', '176cm',
       '177cm', '183cm', '180cm', '189cm', '179cm', '195cm', '172cm',
       '182cm', '186cm', '192cm', '165cm', '194cm', '167cm', '196cm',
       '163cm', '190cm', '174cm', '169cm', '171cm', '197cm', '200cm',
       '166cm', '6\'2"', '164cm', '198cm', '6\'3"', '6\'5"', '5\'11"',
       '6\'4"', '6\'1"', '6\'0"', '5\'10"', '5\'9"', '5\'6"', '5\'7"',
       '5\'4"', '201cm', '158cm', '162cm', '161cm', '160cm', '203cm',
       '157cm', '156cm', '202cm', '159cm', '206cm', '155cm'], dtype=object)

In [222]:
# função que transforma pés em centímetros
def ft_to_cm(x):
    if "'" in x:
        parts = x.split("'")  # Split no '
        feet = int(parts[0])  # parte 1 representa pés
        # parte 2 representa polegadas. retirar " e converter
        inches = int(parts[1].replace('"', '')) if parts[1] else 0 
        
        # calcular cms
        return int(round((feet * 30.48) + (inches * 2.54), 0))
    
    # se já estiver, só faz o strip da string cm e converte
    return int(x.strip("cm"))

df["height"]=df["height"].apply(ft_to_cm)
df["height"].unique()

array([170, 187, 188, 181, 175, 184, 191, 178, 193, 185, 199, 173, 168,
       176, 177, 183, 180, 189, 179, 195, 172, 182, 186, 192, 165, 194,
       167, 196, 163, 190, 174, 169, 171, 197, 200, 166, 164, 198, 201,
       158, 162, 161, 160, 203, 157, 156, 202, 159, 206, 155])

In [223]:
# da mesma forma, peso tem a medida em texto logo depois, como kg ou lbs
print(df["weight"].head())

# retirando a medida kg
# df["weight"]=df["weight"].str.strip("kg")

# mostrando que ainda existem pesos em libras. vamos transformar em kg
df["weight"].unique()

0    72kg
1    83kg
2    87kg
3    70kg
4    68kg
Name: weight, dtype: object


array(['72kg', '83kg', '87kg', '70kg', '68kg', '80kg', '71kg', '91kg',
       '73kg', '85kg', '92kg', '69kg', '84kg', '96kg', '81kg', '82kg',
       '75kg', '86kg', '89kg', '74kg', '76kg', '64kg', '78kg', '90kg',
       '66kg', '60kg', '94kg', '79kg', '67kg', '65kg', '59kg', '61kg',
       '93kg', '88kg', '97kg', '77kg', '62kg', '63kg', '95kg', '100kg',
       '58kg', '183lbs', '179lbs', '172lbs', '196lbs', '176lbs', '185lbs',
       '170lbs', '203lbs', '168lbs', '161lbs', '146lbs', '130lbs',
       '190lbs', '174lbs', '148lbs', '165lbs', '159lbs', '192lbs',
       '181lbs', '139lbs', '154lbs', '157lbs', '163lbs', '98kg', '103kg',
       '99kg', '102kg', '56kg', '101kg', '57kg', '55kg', '104kg', '107kg',
       '110kg', '53kg', '50kg', '54kg', '52kg'], dtype=object)

In [224]:
# função que transforma libras em quilos
def lbs_to_kg(x):
    if "lbs" in x:
        lbs = x.replace("lbs", "")
        kilograms = round(int(lbs) / 2.2, 0)
        return int(kilograms)
    else:
        x_strip = x.strip("kg")
        return int(x_strip)

df["weight"]=df["weight"].apply(lbs_to_kg)
df["weight"].unique()

array([ 72,  83,  87,  70,  68,  80,  71,  91,  73,  85,  92,  69,  84,
        96,  81,  82,  75,  86,  89,  74,  76,  64,  78,  90,  66,  60,
        94,  79,  67,  65,  59,  61,  93,  88,  97,  77,  62,  63,  95,
       100,  58,  98, 103,  99, 102,  56, 101,  57,  55, 104, 107, 110,
        53,  50,  54,  52])

In [225]:
# alguns detalhes acerca das colunas monetárias
df[['value', 'wage', 'release_clause']].head()

,value,wage,release_clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [226]:
# vamos criar uma função para padronizar os valores para euros
def money(x):
    if "€" in x:
        x = x.replace("€", "") # se for euro, substitui por vazio

    if "M" in x:
        x = x.replace("M", "")
        return int(float(x) * 1_000_000) # se tiver M significa que é milhão

    elif "K" in x:
        x = x.replace("K", "")
        return int(float(x) * 1_000) # se tiver K significa que é mil

    return int(x)

# tratamentos finais e renaming
df["value"] = df["value"].apply(money) / 1_000_000
df["wage"] = df["wage"].apply(money)
df["release_clause"] = df["release_clause"].apply(money) / 1_000_000

# renaming
df.rename(columns={
    "value": "values_in_euro_million",
    "wage": "wage_in_euros",
    "release_clause": "release_clause_in_euro_million"
}, inplace=True)

df[['values_in_euro_million', 'wage_in_euros', 'release_clause_in_euro_million']].head()

,values_in_euro_million,wage_in_euros,release_clause_in_euro_million
0,103.5,560000,138.4
1,63.0,220000,75.9
2,120.0,125000,159.4
3,129.0,370000,161.0
4,132.0,270000,166.5


In [227]:
# wf significa Weak Foot (pé fraco, o pé esquerdo de um destro)
# sm significa Skill Moves (dribles)
# ir significa Internation Reputation (reputação internacional)
df[['wf','sm','ir']].head()

,wf,sm,ir
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [228]:
# vamos retirar as estrelas
def remove_star_from_columns(df, columns):
    for col in columns:
        df[col] = df[col].str.replace("★", "", regex=False)
    return df

df = remove_star_from_columns(df, ['wf','sm','ir'])
df.rename(columns={'wf':'wf_rating','sm':'sm_rating','ir':'ir_rating'},inplace =True)
df[['wf_rating','sm_rating','ir_rating']].head()

,wf_rating,sm_rating,ir_rating
0,4,4,5
1,4,5,5
2,3,1,3
3,5,4,4
4,5,5,5


In [229]:
# margem de contrato
df.contract.unique()

array(['2004 ~ 2021', '2018 ~ 2022', '2014 ~ 2023', '2015 ~ 2023',
       '2017 ~ 2022', '2017 ~ 2023', '2018 ~ 2024', '2014 ~ 2022',
       '2018 ~ 2023', '2016 ~ 2023', '2013 ~ 2023', '2011 ~ 2023',
       '2009 ~ 2022', '2005 ~ 2021', '2011 ~ 2021', '2015 ~ 2022',
       '2017 ~ 2024', '2010 ~ 2024', '2012 ~ 2021', '2019 ~ 2024',
       '2015 ~ 2024', '2017 ~ 2025', '2020 ~ 2025', '2019 ~ 2023',
       '2008 ~ 2023', '2015 ~ 2021', '2020 ~ 2022', '2012 ~ 2022',
       '2016 ~ 2025', '2013 ~ 2022', '2011 ~ 2022', '2012 ~ 2024',
       '2016 ~ 2021', '2012 ~ 2023', '2008 ~ 2022', '2019 ~ 2022',
       '2017 ~ 2021', '2013 ~ 2024', '2020 ~ 2024', '2010 ~ 2022',
       '2020 ~ 2021', '2011 ~ 2024', '2020 ~ 2023', '2014 ~ 2024',
       '2013 ~ 2026', '2016 ~ 2022', '2010 ~ 2021', '2013 ~ 2021',
       '2019 ~ 2025', '2018 ~ 2025', '2016 ~ 2024', '2018 ~ 2021',
       '2009 ~ 2024', '2007 ~ 2022', 'Jun 30, 2021 On Loan',
       '2009 ~ 2021', '2019 ~ 2021', '2019 ~ 2026', 'Free', '2012 ~ 

In [230]:
# vamos criar uma função que organiza as informações de início e fim de contrato, além do tipo de contrato
def type(x):
    if "Free" in x:
        return "Free"
    if "Loan" in x:
        return "Loan"
    if "~" in x:
        return "Contract"
    else:
        return pd.na

# função que extrai o início do contrato da string
def start_time_contract(x):
    if "~" in x:
        return  int(x[:4])
    if "Loan" in x:
        x=x.strip(" On Loan")
        x= datetime.strptime(x, "%b %d, %Y")
        return x.date()
    else:
        return "No Club"

# função que extrai o fim do contrato da string
def end_time_contract(type,contract,loan):
    if type == "Contract":
        return int(contract[-4:])
    if type == "Loan":
        loan = datetime.strptime(loan, "%b %d, %Y")
        return loan.date()

    else:
        return "No Club"

# aplicando funções em novas colunas
df["type_of_contract"] = df["contract"].apply(type)
df["start_year"] = df["contract"].apply(start_time_contract)
df["end_year"] = df.apply(lambda row: end_time_contract(row["type_of_contract"],row["contract"],row["loan_date_end"] ), axis=1)

# agora, a coluna loan_date_end é inútil, pois já temos o end_year que a representa
df = df.drop(columns=["loan_date_end"])

# existem jogadores sem clube no momento, consequentemente sem contrato. vamos organizar isso também, preenchendo os nulos de start_year e end__year
# df.fillna({'start_year': "No Club"}, inplace=True)
# df.fillna({'end_year': "No Club"}, inplace=True)

# como ficou as colunas de tipo de contrato, ano de início e ano de fim de contrato
df[['type_of_contract','start_year','end_year']].head()

,type_of_contract,start_year,end_year
0,Contract,2004,2021
1,Contract,2018,2022
2,Contract,2014,2023
3,Contract,2015,2023
4,Contract,2017,2022


In [231]:
# analisando agora as colunas de habilidades dos jogadores
# no fifa, os valores são entre 0 e 100. nota-se que esses valores estão fora do esperado, então vamos normalizá-los
df[['attacking','skill','movement','power','mentality','defending','goalkeeping']].head()

,attacking,skill,movement,power,mentality,defending,goalkeeping
0,429,470,451,389,347,91,54
1,437,414,431,444,353,84,58
2,95,109,307,268,140,57,437
3,407,441,398,408,408,186,56
4,408,448,453,357,356,94,59


In [232]:
# normalizando e transformando em int
df["attacking"] = df["attacking"]/5
df["attacking"]=df["attacking"].astype("int64")

df["skill"] = df["skill"]/5
df["skill"]=df["skill"].astype("int64")


df["movement"]=df["movement"]/5
df["movement"]=df["movement"].astype("int64")


df["power"]=df["power"]/5
df["power"]=df["power"].astype("int64")


df["mentality"]=df["mentality"]/6
df["mentality"]=df["mentality"].astype("int64")

df["defending"]=df["defending"]/3
df["defending"]=df["defending"].astype("int64")

df["goalkeeping"] = df["goalkeeping"]/5
df["goalkeeping"] = df["goalkeeping"].astype("int64")

df = df.rename(columns={"attacking":"attacking_avg", "skill":"skill_avg", "movement":"movement_avg",
                    "power":"power_avg", "mentality":"mentality_avg", "defending":"defending_avg", "goalkeeping":"goalkeeping_avg"} )

df[['attacking_avg','skill_avg','movement_avg','power_avg','mentality_avg','defending_avg','goalkeeping_avg']].head()


,attacking_avg,skill_avg,movement_avg,power_avg,mentality_avg,defending_avg,goalkeeping_avg
0,85,94,90,77,57,30,10
1,87,82,86,88,58,28,11
2,19,21,61,53,23,19,87
3,81,88,79,81,68,62,11
4,81,89,90,71,59,31,11


In [233]:
# tratando a coluna chutes em gol -> valores 1000+ estão como 1K
df['hits'].unique()

array(['771', '562', '150', '207', '595', '248', '246', '120', '1.6K',
       '130', '321', '189', '175', '96', '118', '216', '212', '154',
       '205', '202', '339', '408', '103', '332', '86', '173', '161',
       '396', '1.1K', '433', '242', '206', '177', '1.5K', '198', '459',
       '117', '119', '209', '84', '187', '165', '203', '65', '336', '126',
       '313', '124', '145', '538', '182', '101', '45', '377', '99', '194',
       '403', '414', '593', '374', '245', '3.2K', '266', '299', '309',
       '215', '265', '211', '112', '337', '70', '159', '688', '116', '63',
       '144', '123', '71', '224', '113', '168', '61', '89', '137', '278',
       '75', '148', '176', '197', '264', '214', '247', '402', '440',
       '1.7K', '2.3K', '171', '320', '657', '87', '259', '200', '255',
       '253', '196', '60', '97', '85', '169', '256', '132', '239', '166',
       '121', '109', '32', '46', '122', '48', '527', '199', '282', '51',
       '1.9K', '642', '155', '323', '288', '497', '509', '79',

In [234]:
# função que converte os chutes à gol 
def convert_hits(x):
    if pd.isna(x):
        return np.nan
    elif 'K' in str(x):
        return float(x[:-1])
    else:
        return float(x)

df['hits_in_k'] = df['hits'].apply(convert_hits)

# coluna hits é desnecessária agora
df.drop(columns=['hits'], inplace=True)

# preenchendo os valores vazios pela média de chutes à gol
df.fillna({'hits_in_k': df['hits_in_k'].mean()}, inplace=True)


### Fim do Data Cleaning: Resultados

In [235]:
# colunas renomeadas e seguindo padrão lower caps e _ no lugar dos espaços
df.columns

Index(['id', 'name', 'nationality', 'age', 'ova', 'pot', 'club', 'contract',
       'positions', 'height', 'weight', 'preferred_foot', 'bov',
       'best_position', 'joined', 'values_in_euro_million', 'wage_in_euros',
       'release_clause_in_euro_million', 'attacking_avg', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'skill_avg', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'movement_avg', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'power_avg', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'mentality_avg', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'composure',
       'defending_avg', 'marking', 'standing_tackle', 'sliding_tackle',
       'goalkeeping_avg', 'gk_diving', 'gk_handling', 'gk_kicking',
       'gk_positioning', 'gk_reflexes', 'total_stats', 'base_stats',
       'wf_rating', 'sm_rating', 'aw', 'dw', 'ir_rating', 'p

In [236]:
# preview do dataset
df.head()

,id,name,nationality,age,ova,pot,club,contract,positions,height,...,pac,sho,pas,dri,def,phy,type_of_contract,start_year,end_year,hits_in_k
0,158023,L. Messi,Argentina,33,93,93,FC Barcelona,2004 ~ 2021,"RW, ST, CF",170,...,85,92,91,95,38,65,Contract,2004,2021,771.0
1,20801,Cristiano Ronaldo,Portugal,35,92,92,Juventus,2018 ~ 2022,"ST, LW",187,...,89,93,81,89,35,77,Contract,2018,2022,562.0
2,200389,J. Oblak,Slovenia,27,91,93,Atlético Madrid,2014 ~ 2023,GK,188,...,87,92,78,90,52,90,Contract,2014,2023,150.0
3,192985,K. De Bruyne,Belgium,29,91,91,Manchester City,2015 ~ 2023,"CAM, CM",181,...,76,86,93,88,64,78,Contract,2015,2023,207.0
4,190871,Neymar Jr,Brazil,28,91,91,Paris Saint-Germain,2017 ~ 2022,"LW, CAM",175,...,91,85,86,94,36,59,Contract,2017,2022,595.0


In [237]:
# tratamos os nulos também
df.isna().sum()

id                  0
name                0
nationality         0
age                 0
ova                 0
                   ..
phy                 0
type_of_contract    0
start_year          0
end_year            0
hits_in_k           0
Length: 76, dtype: int64

## Carregamento: Injetando Dataset do Banco de Dados PostgresQL

In [238]:
import json
import psycopg2
import sqlalchemy
from psycopg2 import sql
from sqlalchemy import create_engine

In [239]:
# carregar o arquivo JSON com as configurações de conexão
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# conectando no banco postgreSQL
connection = psycopg2.connect(
    host=config['host'],
    port=config['port'],
    database=config['database'],
    user=config['user'],
    password=config['password']
)

In [240]:
# Conectar o psycopg2 com o SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}', creator=lambda: connection)

# Inserir o DataFrame no banco de dados
df.to_sql('ETL_clean_fifa_model', engine, schema='analytics', if_exists='replace', index=False)

print("DataFrame injetado com sucesso!")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

DataFrame injetado com sucesso!
Tempo total de execução: 6.24 segundos
